In [ ]:
import dash
import dash_auth
from dash import dcc, html, Input, Output, State, callback_context
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import base64
import io
import flask
import pickle
import os

# Librerías para ML
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# ==============================================================================
# 1. CARGA DE DATOS PRE-PROCESADOS (CACHE)
# ==============================================================================
ARCHIVO_CACHE = "datos_dashboard_v12.pkl"
DATA_CACHE = {}

if os.path.exists(ARCHIVO_CACHE):
    print(f"✅ Cargando datos desde '{ARCHIVO_CACHE}'...")
    with open(ARCHIVO_CACHE, "rb") as f:
        DATA_CACHE = pickle.load(f)
else:
    print("⚠️ No se encontró archivo de caché. Se iniciará vacío.")

# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================
VALID_USERNAME_PASSWORD = {"admin": "1234"}

METAS_POR_CLUSTER = {
    "CRÍTICO": 95.0,    
    "MODERADO": 90.0,   
    "ESTABLE": 93.0,    
    "DESCONOCIDO": 98.0 
}

PALETA_KM = ["#868d8e", "#ec6c25", "#e7e5e5", "#e4b193", "#fcd2b9"]

COLORS = {
    "orange": "#ec6c25",      
    "green": "#28a745",       
    "yellow": "#ffc107",      
    "dark_blue": "#2c3e50", 
    "bg_light": "#f4f6f9",
    "text_grey": "#6c757d", 
    "card_grey": "#868d8e"
}

SIDEBAR_STYLE = {
    "position": "fixed", "top": 0, "left": 0, "bottom": 0,
    "width": "18rem", "padding": "2rem 1rem", "backgroundColor": "#f8f9fa",
    "overflowY": "auto", "zIndex": 1000
}

CONTENT_STYLE = {
    "marginLeft": "19rem", "marginRight": "2rem", "padding": "2rem 1rem",
    "backgroundColor": COLORS["bg_light"], "minHeight": "100vh"
}

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.BOOTSTRAP], suppress_callback_exceptions=True)
app.title = "Logistica del Mayab AI"
server = app.server
app.server.secret_key = 'super-secret-key-logimayab'
auth = dash_auth.BasicAuth(app, VALID_USERNAME_PASSWORD)

# ==============================================================================
# COMPONENTES
# ==============================================================================
def create_kpi_card(title, value, id_component=None, bg_color=COLORS["card_grey"]):
    return dbc.Card([
        dbc.CardBody([
            html.H6(title, className="card-title text-white small"),
            html.H3(value, id=id_component if id_component else "", className="card-text text-white fw-bold")
        ])
    ], style={"background-color": bg_color, "border": "none", "border-radius": "15px"}, className="shadow-sm h-100")

def fig_empty(title="Esperando datos..."):
    fig = go.Figure()
    fig.update_layout(
        title=dict(text=title, font=dict(color="gray")),
        xaxis={"visible": False}, yaxis={"visible": False}, 
        paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', 
        height=350, margin=dict(l=20, r=20, t=40, b=20)
    )
    return fig

def upload_component(id_name, label_text):
    return html.Div([
        html.Label(label_text, className="fw-bold mb-2", style={"fontSize": "0.85rem"}),
        dcc.Upload(
            id=id_name,
            children=html.Div(['Arrastra o ', html.A('Selecciona Excel')]),
            style={
                'width': '100%', 'height': '50px', 'lineHeight': '50px',
                'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
                'textAlign': 'center', 'backgroundColor': 'white', 'cursor': 'pointer',
                'fontSize': '0.8rem'
            }, multiple=False
        ),
        html.Div(id=f"{id_name}-output", className="mt-1 small text-muted")
    ], className="mb-3")

# ==============================================================================
# LAYOUTS
# ==============================================================================

layout_upload = html.Div([
    html.H2("Configuración de Datos", className="mb-4", style={"color": COLORS["dark_blue"], "fontWeight": "bold"}),
    html.P("Carga los archivos operativos para alimentar el modelo de IA y los KPIs.", className="text-muted mb-4"),
    
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("1. Combustible", className="text-white small", style={"backgroundColor": COLORS["dark_blue"]}),
                dbc.CardBody(upload_component('upload-combustible', 'Reporte Combustible (.xlsx)'))
            ], className="h-100 shadow-sm")
        ], md=4),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("2. Operaciones (Rutas & IA)", className="text-white small", style={"backgroundColor": COLORS["orange"]}),
                dbc.CardBody([
                    html.P("Alimenta catálogo, rutas y simulación.", className="small text-muted mb-2"),
                    upload_component('upload-viajes', 'Reporte Listado Viajes (.xlsx)'),
                    html.Div(id='upload-viajes-output', className="small fw-bold text-center mt-2"),
                    html.Div(id='cluster-calc-output', style={'display': 'none'}) 
                ])
            ], className="h-100 shadow-sm")
        ], md=4),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("3. KPIs Financieros", className="text-white small", style={"backgroundColor": COLORS["dark_blue"]}),
                dbc.CardBody([
                    html.P("Calcula CPK y Costo Parado.", className="small text-muted mb-2"),
                    upload_component('upload-telemetria', 'A. Telemetría'),
                    upload_component('upload-tablero', 'B. Tablero PL'),
                    html.Div(id='upload-financiero-output', className="text-center fw-bold mt-2 small"),
                    html.Div(id='cpk-calc-output', style={'display': 'none'}),
                    html.Div(id='ociosidad-calc-output', style={'display': 'none'})
                ])
            ], className="h-100 shadow-sm")
        ], md=4),
    ]),
    dbc.Row([
        dbc.Col([
            html.Hr(),
            dbc.Button("Ir al Dashboard ->", href="/dashboard", style={"backgroundColor": COLORS["orange"], "border": "none"}, size="lg", className="mt-3")
        ], width=12, className="text-end")
    ])
])

layout_dashboard = html.Div([
    dbc.Row([dbc.Col(html.Div([html.I(className="bi bi-layout-sidebar me-3"), html.Span("Inicio / ", className="text-muted"), html.Span("Dashboard Operativo", className="fw-bold")], className="d-flex align-items-center mb-4"))]),
    
    dbc.Row([
        dbc.Col(create_kpi_card("Indicador Ociosidad", "---", id_component="kpi-ociosidad", bg_color=COLORS["orange"]), md=3),
        dbc.Col(create_kpi_card("Precio Diesel", "---", id_component="kpi-combustible"), md=3),
        dbc.Col(create_kpi_card("CPK Promedio", "---", id_component="kpi-cpk", bg_color=COLORS["orange"]), md=3),
        dbc.Col(create_kpi_card("Nivel de Servicio Meta", "98%", id_component="kpi-nivel-servicio"), md=3),
    ], className="mb-4"),
    
    dbc.Row([
        dbc.Col([dbc.Card([dbc.CardBody([html.H6("ESTADO DE LA RUTA (CLÚSTER IA)", className="small text-muted mb-3 text-center"), html.Div([html.Div("---", id="cluster-circle-text", className="d-flex align-items-center justify-content-center fw-bold shadow-sm", style={"width": "120px", "height": "120px", "background-color": COLORS["card_grey"], "border-radius": "50%", "margin": "0 auto", "color": "white", "font-size": "1.1rem", "transition": "0.3s"})], className="text-center mb-4"), html.P("Estrategia Recomendada:", className="text-center text-muted small mb-1"), html.P("Seleccione una ruta o cargue datos.", id="cluster-strategy-text", className="text-center fw-bold small text-dark")])], className="h-100 shadow-sm border-0")], md=3),
        dbc.Col([dbc.Card(dbc.CardBody(dcc.Graph(id='grafico-tradeoff', figure=fig_empty("Esperando selección de ruta..."))), className="h-100 shadow-sm border-0")], md=9),
    ], className="mb-4"),
    
    dbc.Row([
        dbc.Col([dbc.Card(dbc.CardBody(dcc.Graph(id='grafico-km', figure=fig_empty("Datos de KM pendientes..."))), className="shadow-sm border-0")], md=6),
        dbc.Col([dbc.Card(dbc.CardBody(dcc.Graph(id='grafico-buffer', figure=fig_empty("Cálculo de Buffer pendiente..."))), className="shadow-sm border-0")], md=6),
    ])
])

sidebar = html.Div([
    html.Div([html.Img(src="[https://logimayab.com.mx/wp-content/uploads/2022/12/icono-logi-mayab.png](https://logimayab.com.mx/wp-content/uploads/2022/12/icono-logi-mayab.png)", style={'height': '40px', 'width': 'auto', 'marginRight': '10px'}), html.H4("LogiMayab", style={'display': 'inline-block', 'verticalAlign': 'middle'})], style={'marginBottom': '20px'}),
    html.Hr(),
    html.Div(id="user-display", className="mb-3 fw-bold text-center small"),
    dbc.Nav([dbc.NavLink([html.I(className="bi bi-upload me-2"), "Carga de Datos"], href="/", active="exact"), dbc.NavLink([html.I(className="bi bi-grid me-2"), "Dashboard"], href="/dashboard", active="exact")], vertical=True, pills=True, className="mb-4"),
    html.Div([html.Label("Filtro de Ruta:", className="text-muted small fw-bold mb-1"), dcc.Dropdown(id='sidebar-ruta-dropdown', placeholder="Cargue rutas primero...", style={'fontSize': '0.85rem'})], className="p-2 bg-white rounded border mt-auto")
], style=SIDEBAR_STYLE)

# --- APP LAYOUT ---
# Inicializamos los stores con los datos del CACHE si existen
app.layout = html.Div([
    dcc.Location(id="url"),
    dcc.Store(id='store-combustible-price', storage_type='session', data=DATA_CACHE.get('combustible_price')),
    dcc.Store(id='store-rutas-options', storage_type='session', data=DATA_CACHE.get('rutas_options')),
    dcc.Store(id='store-telemetria-data', storage_type='session', data=DATA_CACHE.get('telemetria_data')),
    dcc.Store(id='store-tablero-data', storage_type='session', data=DATA_CACHE.get('tablero_data')),
    dcc.Store(id='store-cpk-final', storage_type='session', data=DATA_CACHE.get('cpk_final')),
    dcc.Store(id='store-cluster-results', storage_type='session', data=DATA_CACHE.get('cluster_results')),
    dcc.Store(id='store-ociosidad-global', storage_type='session', data=DATA_CACHE.get('ociosidad_global')),
    dcc.Store(id='store-viajes-processed', storage_type='session', data=DATA_CACHE.get('viajes_processed')),
    
    sidebar,
    html.Div(id="page-content", style=CONTENT_STYLE)
])

app.validation_layout = html.Div([layout_upload, layout_dashboard, sidebar, app.layout])

# ==============================================================================
# CALLBACKS
# ==============================================================================

def find_period_for_date(date, periods_info):
    if pd.isna(date): return None
    try: date = pd.to_datetime(date)
    except: return None
    matching = periods_info[(periods_info['Fecha de inicio del periodo'] <= date) & (periods_info['Fecha de fin del periodo'] >= date)]
    if not matching.empty: return matching['Periodo'].iloc[0]
    return None

@app.callback(Output("page-content", "children"), Output("user-display", "children"), [Input("url", "pathname")])
def render_page(pathname):
    username = flask.request.authorization['username'] if flask.request.authorization else "Admin"
    if pathname == "/dashboard": return layout_dashboard, f"Usuario: {username}"
    return layout_upload, f"Usuario: {username}"

# --- UPLOADS (Solo si el usuario carga algo nuevo) ---
@app.callback([Output('store-viajes-processed', 'data'), Output('store-rutas-options', 'data'), Output('upload-viajes-output', 'children')], Input('upload-viajes', 'contents'))
def process_viajes(contents):
    if not contents: return dash.no_update, dash.no_update, dash.no_update
    try:
        df = pd.read_excel(io.BytesIO(base64.b64decode(contents.split(',')[1])))
        if 'Viaje' not in df.columns: df['Viaje'] = 1 
        cols_needed = ['Ruta', 'Fecha Salida', 'Fecha Llegada', 'Estatus de Viaje', 'Nro Ope', 'Operador', 'Tractocamión', 'Viaje', 'Nombre Cliente']
        df = df[[c for c in cols_needed if c in df.columns]].drop_duplicates()
        rutas = [{'label': str(r), 'value': str(r)} for r in df['Ruta'].dropna().unique()]
        data_dict = df.to_dict('records')
        return data_dict, rutas, html.Span(f"✅ {len(df)} viajes procesados", className="text-success")
    except Exception as e: return dash.no_update, dash.no_update, html.Span(f"❌ Error: {str(e)}", className="text-danger")

@app.callback(Output('store-cluster-results', 'data'), Input('store-viajes-processed', 'data'))
def run_cluster_model(data):
    if not data: return dash.no_update
    try:
        df = pd.DataFrame(data)
        df['Fecha Salida'] = pd.to_datetime(df['Fecha Salida'], errors='coerce')
        df['Fecha Llegada'] = pd.to_datetime(df['Fecha Llegada'], errors='coerce')
        df = df.dropna(subset=['Fecha Salida', 'Estatus de Viaje']).copy()
        df = df[df['Fecha Salida'].dt.year >= 2024]
        ruta_volumen = df.groupby('Ruta')['Viaje'].count().reset_index(name='Total_Viajes')
        ruta_cliente = df.groupby(['Ruta', 'Nombre Cliente'])['Viaje'].count().reset_index()
        ruta_max_cliente = ruta_cliente.sort_values(['Ruta', 'Viaje'], ascending=[True, False]).groupby('Ruta').first().reset_index()
        data_cluster = pd.merge(ruta_volumen, ruta_max_cliente[['Ruta', 'Viaje']], on='Ruta')
        data_cluster['Porcentaje_Dependencia'] = data_cluster['Viaje'] / data_cluster['Total_Viajes']
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(data_cluster[['Total_Viajes', 'Porcentaje_Dependencia']])
        kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
        data_cluster['Cluster_Riesgo'] = kmeans.fit_predict(X_scaled)
        analysis = data_cluster.groupby('Cluster_Riesgo')['Porcentaje_Dependencia'].mean().sort_values(ascending=False)
        id_critico, id_estable = analysis.index[0], analysis.index[-1]
        id_moderado = [x for x in [0,1,2] if x not in [id_critico, id_estable]][0]
        mapa = {int(id_critico): "CRÍTICO", int(id_moderado): "MODERADO", int(id_estable): "ESTABLE"}
        res = {}
        for _, row in data_cluster.iterrows(): res[row['Ruta']] = mapa.get(int(row['Cluster_Riesgo']), "DESCONOCIDO")
        return res
    except: return dash.no_update

@app.callback([Output('store-combustible-price', 'data'), Output('upload-combustible-output', 'children')], Input('upload-combustible', 'contents'))
def process_fuel(c):
    if not c: return dash.no_update, dash.no_update
    try:
        df = pd.read_excel(io.BytesIO(base64.b64decode(c.split(',')[1])))
        if 'Precio Unitario Merc' in df.columns:
            df['Precio Unitario Merc'] = pd.to_numeric(df['Precio Unitario Merc'].astype(str).str.replace(r'[$,]', '', regex=True), errors='coerce')
            val = df[df['Precio Unitario Merc'] > 0]['Precio Unitario Merc'].mean()
            return val, html.Span(f"✅ ${val:.2f}", className="text-success")
    except: pass
    return dash.no_update, html.Span("❌ Error", className="text-danger")

@app.callback([Output('store-telemetria-data', 'data'), Output('store-tablero-data', 'data'), Output('upload-financiero-output', 'children')], [Input('upload-telemetria', 'contents'), Input('upload-tablero', 'contents')])
def process_finance(tele_con, tabl_con):
    if not tele_con and not tabl_con: return dash.no_update, dash.no_update, dash.no_update
    tele_data, tabl_data, msg = dash.no_update, dash.no_update, []
    if tele_con:
        try:
            df_tele = pd.read_excel(io.BytesIO(base64.b64decode(tele_con.split(',')[1])))
            cols_tele = ['Nombre', 'Periodo', 'Distancia', 'Fecha de inicio del periodo', 'Fecha de fin del periodo']
            tele_data = df_tele[[c for c in cols_tele if c in df_tele.columns]].to_dict('records')
            msg.append("✅ Tele")
        except: msg.append("❌ Tele")
    if tabl_con:
        try:
            df_pl = pd.read_excel(io.BytesIO(base64.b64decode(tabl_con.split(',')[1])))
            tabl_data = df_pl[['Rubro', 'Date', 'Real']].to_dict('records')
            msg.append("✅ Tablero")
        except: msg.append("❌ Tablero")
    return tele_data, tabl_data, " | ".join(msg)

@app.callback([Output('store-cpk-final', 'data'), Output('store-ociosidad-global', 'data')], [Input('store-telemetria-data', 'data'), Input('store-tablero-data', 'data')])
def calc_derived_kpis(tele_data, tabl_data):
    if not tele_data or not tabl_data: return dash.no_update, dash.no_update
    try:
        df_tele = pd.DataFrame(tele_data)
        df_pl = pd.DataFrame(tabl_data)
        df_tele['Fecha de inicio del periodo'] = pd.to_datetime(df_tele['Fecha de inicio del periodo'])
        df_tele['Mes'] = df_tele['Fecha de inicio del periodo'].dt.to_period('M').astype(str)
        df_pl['Date'] = pd.to_datetime(df_pl['Date'])
        df_pl['Mes'] = df_pl['Date'].dt.to_period('M').astype(str)
        dist_mes = df_tele.groupby('Mes')['Distancia'].sum().reset_index()
        cost_mes = df_pl[df_pl["Rubro"] == "2. Costo"][["Mes", "Real"]]
        merged = pd.merge(cost_mes, dist_mes, on="Mes")
        cpk = (merged["Real"] / merged["Distancia"]).mean()
        unidades = len(df_tele["Nombre"].unique())
        ultimo_mes = df_pl['Date'].max()
        rubros = ['Salarios', 'Mantenimiento Automotriz', 'Depreciación', 'Arrendamiento']
        costo_fijo = df_pl[(df_pl['Rubro'].isin(rubros)) & (df_pl['Date'] == ultimo_mes)]['Real'].sum()
        ociosidad = (costo_fijo / unidades / 30) if unidades > 0 else 0
        return cpk, ociosidad
    except: return dash.no_update, dash.no_update

# --- GRAFICAS Y UI ---
@app.callback(Output('kpi-combustible', 'children'), Input('store-combustible-price', 'data'))
def up_fuel(p): return f"${p:,.2f}" if p else "$0.00"

@app.callback(Output('kpi-cpk', 'children'), Input('store-cpk-final', 'data'))
def up_cpk(p): return f"${p:,.2f}" if p else "---"

@app.callback([Output('sidebar-ruta-dropdown', 'options'), Output('sidebar-ruta-dropdown', 'placeholder')], Input('store-rutas-options', 'data'))
def up_sidebar(o): return (o, "Seleccione ruta...") if o else ([], "Cargue Operaciones...")

@app.callback([Output('grafico-tradeoff', 'figure'), Output('kpi-ociosidad', 'children')], [Input('sidebar-ruta-dropdown', 'value'), Input('store-viajes-processed', 'data'), Input('store-cluster-results', 'data'), Input('store-ociosidad-global', 'data')])
def update_tradeoff(ruta, viajes_data, cluster_res, costo_global):
    fig_def = fig_empty("Seleccione ruta")
    kpi_def = f"${costo_global:,.2f}/día" if costo_global else "---"
    if not ruta or not viajes_data: return fig_def, kpi_def
    try:
        estado = cluster_res.get(ruta, "DESCONOCIDO") if cluster_res else "DESCONOCIDO"
        meta_ns = METAS_POR_CLUSTER.get(estado, 98.0)
        df = pd.DataFrame(viajes_data)
        df['Fecha Salida'] = pd.to_datetime(df['Fecha Salida'])
        df_ruta = df[df['Ruta'] == ruta].copy()
        if len(df_ruta) < 2: return fig_empty(f"Datos insuficientes para {ruta}"), kpi_def
        demanda = df_ruta.set_index('Fecha Salida').resample('W')['Viaje'].count().fillna(0)
        cap_base = int(np.ceil(demanda.mean()))
        res_curva, optimo = [], None
        for buff in range(0, int(demanda.max()) + 5):
            cap = cap_base + buff
            atendidos = np.minimum(demanda, cap)
            ociosas = np.maximum(0, cap - demanda)
            ns = (atendidos.sum() / demanda.sum() * 100) if demanda.sum() > 0 else 0
            cap_disp = cap * len(demanda)
            tasa_ocio = (ociosas.sum() / cap_disp * 100) if cap_disp > 0 else 0
            pt = {'Flota': cap, 'NS': ns, 'Ocio': tasa_ocio}
            res_curva.append(pt)
            if optimo is None and ns >= meta_ns: optimo = pt
        if optimo is None: optimo = res_curva[-1]
        df_res = pd.DataFrame(res_curva)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_res['Flota'], y=df_res['NS'], name='Nivel Servicio', line=dict(color='#1f77b4', width=3)))
        fig.add_trace(go.Scatter(x=df_res['Flota'], y=df_res['Ocio'], name='Ociosidad', yaxis='y2', line=dict(color='#ff7f0e', dash='dot', width=3)))
        fig.add_shape(type="line", x0=df_res['Flota'].min(), x1=df_res['Flota'].max(), y0=meta_ns, y1=meta_ns, line=dict(color="green", dash="dash"))
        fig.add_shape(type="line", x0=optimo['Flota'], x1=optimo['Flota'], y0=0, y1=100, line=dict(color="gray", width=1), yref='paper')
        fig.update_layout(title=f"Simulación: {ruta}", xaxis_title="Flota Total", yaxis=dict(title="NS (%)", range=[50,105]), yaxis2=dict(title="Ociosidad (%)", overlaying="y", side="right", range=[0,100]), legend=dict(orientation="h", y=1.1), height=350, margin=dict(l=40, r=40, t=40, b=40))
        return fig, f"{optimo['Ocio']:.1f}% (Ruta)"
    except Exception as e: return fig_empty(f"Error: {str(e)}"), kpi_def

@app.callback(Output('grafico-km', 'figure'), [Input('sidebar-ruta-dropdown', 'value'), Input('store-viajes-processed', 'data'), Input('store-telemetria-data', 'data')])
def update_km(ruta, viajes_data, tele_data):
    if not ruta or not viajes_data or not tele_data: return fig_empty("Faltan datos")
    try:
        df_viajes = pd.DataFrame(viajes_data)
        df_viajes['Fecha Salida'] = pd.to_datetime(df_viajes['Fecha Salida'])
        df_tele = pd.DataFrame(tele_data)
        df_tele['Fecha de inicio del periodo'] = pd.to_datetime(df_tele['Fecha de inicio del periodo'])
        df_tele['Fecha de fin del periodo'] = pd.to_datetime(df_tele['Fecha de fin del periodo'])
        periodos_info = df_tele[['Periodo', 'Fecha de inicio del periodo', 'Fecha de fin del periodo']].drop_duplicates()
        km_base = df_tele.groupby(['Nombre', 'Periodo'])['Distancia'].sum().reset_index()
        trips = df_viajes[df_viajes['Ruta'] == ruta].copy()
        if trips.empty: return fig_empty("Sin viajes")
        trips['Periodo'] = trips['Fecha Salida'].apply(lambda x: find_period_for_date(x, periodos_info))
        trips = trips.dropna(subset=['Periodo'])
        trips['Periodo'] = trips['Periodo'].astype(int)
        trips_uniq = trips[['Tractocamión', 'Periodo']].drop_duplicates().rename(columns={'Tractocamión': 'Nombre'})
        merged = pd.merge(trips_uniq, km_base, on=['Nombre', 'Periodo'])
        final = merged.groupby('Periodo')['Distancia'].sum().reset_index().sort_values('Periodo')
        fig = px.bar(final, x='Periodo', y='Distancia', title=f"KM Recorridos: {ruta}", color_discrete_sequence=PALETA_KM)
        fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', height=300, margin=dict(l=20, r=20, t=40, b=20))
        return fig
    except: return fig_empty("Error cálculo KM")

@app.callback(Output('grafico-buffer', 'figure'), [Input('store-viajes-processed', 'data'), Input('store-ociosidad-global', 'data'), Input('store-cluster-results', 'data')])
def update_buffer(viajes_data, costo_diario, clusters):
    if not viajes_data or not costo_diario: return fig_empty("Faltan datos financieros")
    try:
        df = pd.DataFrame(viajes_data)
        df['Fecha Salida'] = pd.to_datetime(df['Fecha Salida'])
        top_rutas = df['Ruta'].value_counts().nlargest(20).index.tolist()
        res = []
        for r in top_rutas:
            demanda = df[df['Ruta']==r].set_index('Fecha Salida').resample('W')['Viaje'].count().fillna(0)
            if demanda.sum() == 0: continue
            estado = clusters.get(r, "DESCONOCIDO") if clusters else "DESCONOCIDO"
            meta = METAS_POR_CLUSTER.get(estado, 98.0)
            base = int(np.ceil(demanda.mean()))
            opt_buff = 0
            for buff in range(0, int(demanda.max())+2):
                cap = base + buff
                ns = (np.minimum(demanda, cap).sum() / demanda.sum() * 100)
                if ns >= meta:
                    opt_buff = buff
                    break
            res.append({'Ruta': r, 'Costo': opt_buff * costo_diario * 30, 'Estado': estado})
        df_res = pd.DataFrame(res).sort_values('Costo', ascending=False).head(15)
        fig = px.bar(df_res, x='Costo', y='Ruta', orientation='h', title=f"Top Costo Buffer (Ocio: ${costo_diario:,.0f}/día)", color='Estado', color_discrete_map={"CRÍTICO": COLORS["orange"], "MODERADO": COLORS["yellow"], "ESTABLE": COLORS["green"]})
        fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', height=300, margin=dict(l=20, r=20, t=40, b=20), yaxis={'categoryorder':'total ascending'})
        return fig
    except: return fig_empty("Error Buffer")

@app.callback(Output('kpi-nivel-servicio', 'children'), [Input('sidebar-ruta-dropdown', 'value'), Input('store-cluster-results', 'data')])
def up_ns_kpi(r, c): 
    return f"{METAS_POR_CLUSTER.get(c.get(r, 'DESCONOCIDO'), 98.0)}%" if r and c else "98%"

@app.callback([Output("cluster-circle-text", "children"), Output("cluster-circle-text", "style"), Output("cluster-strategy-text", "children")], [Input("sidebar-ruta-dropdown", "value"), Input("store-cluster-results", "data")])
def up_cluster_ui(r, c):
    st = {"width": "120px", "height": "120px", "border-radius": "50%", "margin": "0 auto", "color": "white", "display": "flex", "align-items": "center", "justify-content": "center", "fontSize": "1.1rem"}
    if not r or not c: 
        st["background-color"] = COLORS["card_grey"]
        return "---", st, "Seleccione ruta..."
    es = c.get(r, "N/A")
    col = COLORS.get("orange" if es=="CRÍTICO" else "yellow" if es=="MODERADO" else "green" if es=="ESTABLE" else "card_grey")
    st.update({"background-color": col, "color": "black" if es!="ESTABLE" else "white"})
    return es, st, "⚠️ Atención" if es in ["CRÍTICO", "MODERADO"] else "✅ OK"

if __name__ == "__main__":
    app.run(debug=True)